In [15]:
import os
import ee
import numpy as np
from geeml.extract import extractor
import pandas as pd
# import geemap
# Authenticate GEE
# ee.Authenticate()
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/explore/nobackup/people/spotter5/cnn_mapping/gee-serdp-upload-7cd81da3dc69.json"

service_account = 'gee-serdp-upload@appspot.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, "/explore/nobackup/people/spotter5/cnn_mapping/gee-serdp-upload-7cd81da3dc69.json")
ee.Initialize(credentials)
# Initialize GEE with high-volume end-point
# ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')
ee.Initialize()

In [16]:
import geemap
import os
from google.cloud import storage
from google.cloud import client

In [17]:
os.environ["GCLOUD_PROJECT"] = "gee-serdp-upload"

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/explore/nobackup/people/spotter5/cnn_mapping/gee-serdp-upload-7cd81da3dc69.json"
storage_client = storage.Client.from_service_account_json("/explore/nobackup/people/spotter5/cnn_mapping/gee-serdp-upload-7cd81da3dc69.json")

os.environ["GCLOUD_PROJECT"] = "gee-serdp-upload"
storage_client = storage.Client()
# bucket_name = 'smp-scratch/mtbs_1985'
bucket_name = 'smp-scratch'

bucket = storage_client.bucket(bucket_name)

In [18]:
dem = ee.Image("UMN/PGC/ArcticDEM/V3/2m_mosaic") #arctic dem
sent_2A = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
s2Clouds = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
# territory22 = ee.FeatureCollection("users/spotter/alaska") #ak shapefile
# grd =  ee.FeatureCollection("users/spotter/alaska").union()
# territory22 = ee.FeatureCollection("users/spotter/alaska").union()
grd =  ee.FeatureCollection("users/spotter/fire_cnn/raw/ea_grid")
territory22 = ee.FeatureCollection("users/spotter/fire_cnn/raw/ea_grid")

# grd =  ee.FeatureCollection("users/spotter/fire_cnn/raw/nwt_cent")
# territory22 = ee.FeatureCollection("users/spotter/fire_cnn/raw/nwt_cent")

In [19]:
def mask(image):
    qa = image.select('QA_PIXEL')                                       
    mask = qa.bitwiseAnd(8).eq(0).And(qa.bitwiseAnd(10).eq(0)).And(qa.bitwiseAnd(32).eq(0))  
    return(image.updateMask(mask))

def land_scale(image):

    return(image.multiply(0.0000275).add(-0.2))

def sent_scale(image):
    return(image.multiply(0.0001))

In [20]:

#remove clouds from sentinel 2, cloud probability of less than 15%
def sent_maskcloud(image):

    QA60 = image.select(['QA60'])
    clouds = QA60.bitwiseAnd(1<<10).Or(QA60.bitwiseAnd(1<<11))# this gives us cloudy pixels
    image = image.select(['B2', 'B3', 'B4', 'B8', 'B11', 'B12'], ['B1', 'B2', 'B3', 'B4', 'B5', 'B7'])# rename bands
    image =  image.updateMask(clouds.Not()) # remove the clouds from image

    #different resolutions so change to 30 seperately
    image1 = image.select(['B1', 'B2', 'B3', 'B4'], ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4'])
    image2 = image.select(['SR_B5', 'SR_B7'])

    #reproject 30m
    image1 = image1.reproject(
    crs = image1.projection().crs(),
    scale = 30)

    image2 = image2.reproject(
    crs = image2.projection().crs(),
    scale = 30)

    image = image1.addBands(image2)

    return image.toShort()

MAX_CLOUD_PROBABILITY = 50

def sent_maskcloud(image):
    
    clouds = ee.Image(image.get('cloud_mask')).select('probability')
    
    isNotCloud = clouds.lt(MAX_CLOUD_PROBABILITY)
    
    return image.updateMask(isNotCloud)

#Join S2 SR with cloud probability dataset to add cloud mask.
s2SrWithCloudMask = ee.Join.saveFirst('cloud_mask').apply(
    
  primary=sent_2A,
  secondary=s2Clouds,
  condition=ee.Filter.equals(leftField='system:index', rightField='system:index'))

sent_2A = ee.ImageCollection(s2SrWithCloudMask).map(sent_maskcloud)


Functions to apply logan corrections, first pull the necessary intercepts and coeffiterritory22s from logans csv file


In [21]:
import pandas as pd
coeffs = pd.read_csv("/explore/nobackup/people/spotter5/cnn_mapping/raw_files/boreal_xcal_regression_coefficients.csv").fillna(0)
#l5
def landsat_correct(sat, bands):

    """argument 1 is which sattelite, LANDASAT_5 or LANDSAT_8
    argument 2 is bands of interest.  Bands must be in same order as EE,
    
    regression is of form,
    L7 = B0 + (B1 * L5/8) + (B2 * L^2) + (B3 * L^3)
    """

    #bands of interest in order of interest
    l5 = coeffs[(coeffs['satellite'] == sat) & (coeffs['band.or.si'] .isin (bands))] 

    #arrange the band or si column
    l5['band.or.si']=pd.Categorical(l5['band.or.si'],categories=bands)
    l5=l5.sort_values('band.or.si')

    b0 = l5['B0'].values.tolist()
    b1 = l5['B1'].values.tolist()
    b2 = l5['B2'].values.tolist()
    b3 = l5['B3'].values.tolist()

    return (b0, b1, b2, b3)

#get the corrections, each output is a list at one of the four locations
l8_corr = landsat_correct(sat = 'LANDSAT_8', bands = ['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'nbr', 'ndvi', 'ndii'])
l5_corr = landsat_correct(sat = 'LANDSAT_5', bands = ['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'nbr', 'ndvi', 'ndii'])




/tmp/ipykernel_3210599/2309367526.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  l5['band.or.si']=pd.Categorical(l5['band.or.si'],categories=bands)
/tmp/ipykernel_3210599/2309367526.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  l5['band.or.si']=pd.Categorical(l5['band.or.si'],categories=bands)


Read in the file which will normalize to the global values

In [22]:
#first row is min, second is max
# glob_norm = pd.read_csv("/explore/nobackup/people/spotter5/cnn_mapping/nbac_training/l8_sent_collection2_0_global_min_max.csv")
# glob_norm

# glob_norm  = pd.read_csv("/explore/nobackup/people/spotter5/cnn_mapping/nbac_training/l8_sent_collection2_global_min_max_cutoff_negative.csv").reset_index(drop = True)
glob_norm = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/nbac_training/l8_sent_collection2_global_min_max_cutoff_proj.csv').reset_index(drop = True)
# glob_norm = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/nbac_training/l8_sent_collection2_global_min_max_cutoff_proj_pos.csv').reset_index(drop = True)

glob_norm  = glob_norm [['0', '1', '2', '3', '4', '5', '6', '7', '8']]
glob_norm = glob_norm.reset_index()
glob_norm

,index,0,1,2,3,4,5,6,7,8
0,0,-54.0,-49.0,-63.0,-49.0,-129.0,-155.0,21.0,-68.0,-9.0
1,1,44.0,55.0,51.0,175.0,73.0,18.0,974.0,522.0,686.0


Read in the model

In [23]:
def to_float(image):

    b1 = image.select('SR_B1').cast({'SR_B1':'float'}) #0
    b2 = image.select('SR_B2').cast({'SR_B2':'float'}) #1
    b3 = image.select('SR_B3').cast({'SR_B3':'float'}) #2
    b4 = image.select('SR_B4').cast({'SR_B4':'float'}) #3
    b5 = image.select('SR_B5').cast({'SR_B5':'float'}) #4
    b6 = image.select('SR_B7').cast({'SR_B7':'float'}) #5

    image = b1.addBands(b2).addBands(b3).addBands(b4).addBands(b5).addBands(b6)

    return image





In [24]:
def get_pre_post(pre_start, pre_end, post_start, post_end, geometry):

    """parameters are:
    pre_start: the start date for pre fire imagery
    pre_end: the end date for pre fire imagery
    post_start: the start date for post fire imagery
    post_end: the end date for post_fire imagery
    geometry: the geometry to filter by
    """

    #landsat 5
    lt5 = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2').filterDate(pre_start, post_end).filterBounds(geometry)
    #landsat 7
    le7 = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2').filterDate(pre_start, post_end).filterBounds(geometry)
    #landsat 8
    lc8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').filterDate(pre_start, post_end).filterBounds(geometry)
    #sentinel 2
    sent = sent_2A.filterDate(pre_start, post_end).filterBounds(geometry)

    #select bands
    pre_lt5 = lt5.filterDate(pre_start, pre_end).map(mask).map(land_scale).select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7']).map(to_float)

    #       #ensure we have imagery for the sensor
    if pre_lt5.size().getInfo() > 0 :


        #take the median
        pre_lt5 = pre_lt5.median().clip(geometry)

        #calculate nbr, ndvi and ndii
        pre_lt5_nbr = pre_lt5.normalizedDifference(['SR_B4', 'SR_B7']).select([0], ['NBR']).cast({'NBR': 'float'})
        pre_lt5_ndvi = pre_lt5.normalizedDifference(['SR_B4', 'SR_B3']).select([0], ['NDVI']).cast({'NDVI': 'float'})
        pre_lt5_ndii = pre_lt5.normalizedDifference(['SR_B4', 'SR_B5']).select([0], ['NDII']).cast({'NDII': 'float'})

        #add the bands back
        pre_lt5 = pre_lt5.addBands(pre_lt5_nbr).addBands(pre_lt5_ndvi).addBands(pre_lt5_ndii)

        #apply the corrections

        l5_pre_corr = pre_lt5.multiply(l5_corr[1]).add(pre_lt5.pow(2).multiply(l5_corr[2])).add(pre_lt5.pow(3).multiply(l5_corr[3])).add(l5_corr[0])

    #-------now do post-fire
    #select bands
    post_lt5 = lt5.filterDate(post_start, post_end).map(mask).map(land_scale).select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7']).map(to_float)

    #       #ensure we have imagery for the sensor
    if post_lt5.size().getInfo() > 0 :



        #take the median
        post_lt5 = post_lt5.median().clip(geometry)

        #calculate nbr, ndvi and ndii
        post_lt5_nbr = post_lt5.normalizedDifference(['SR_B4', 'SR_B7']).select([0], ['NBR']).cast({'NBR': 'float'})
        post_lt5_ndvi = post_lt5.normalizedDifference(['SR_B4', 'SR_B3']).select([0], ['NDVI']).cast({'NDVI': 'float'})
        post_lt5_ndii = post_lt5.normalizedDifference(['SR_B4', 'SR_B5']).select([0], ['NDII']).cast({'NDII': 'float'})

        #add the bands back
        post_lt5 = post_lt5.addBands(post_lt5_nbr).addBands(post_lt5_ndvi).addBands(post_lt5_ndii)

        #apply the corrections

        l5_post_corr = post_lt5.multiply(l5_corr[1]).add(post_lt5.pow(2).multiply(l5_corr[2])).add(post_lt5.pow(3).multiply(l5_corr[3])).add(l5_corr[0])


      #         #------------------------------------------Landsat 7, no corrections but get things clipped and do pre fire/post_fire stuff


    #select bands
    pre_le7 = le7.filterDate(pre_start, pre_end).map(mask).map(land_scale).select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7']).map(to_float)

    #       #ensure we have imagery for the sensor
    if pre_le7.size().getInfo() > 0 :



        #take the median
        pre_le7 = pre_le7.median().clip(geometry)

        #calculate nbr, ndvi and ndii
        pre_le7_nbr = pre_le7.normalizedDifference(['SR_B4', 'SR_B7']).select([0], ['NBR']).cast({'NBR': 'float'})
        pre_le7_ndvi = pre_le7.normalizedDifference(['SR_B4', 'SR_B3']).select([0], ['NDVI']).cast({'NDVI': 'float'})
        pre_le7_ndii = pre_le7.normalizedDifference(['SR_B4', 'SR_B5']).select([0], ['NDII']).cast({'NDII': 'float'})

        #add the bands back
        pre_le72 = pre_le7.addBands(pre_le7_nbr).addBands(pre_le7_ndvi).addBands(pre_le7_ndii)

    #-------now do post-fire
    #select bands
    post_le7 = le7.filterDate(post_start, post_end).map(mask).map(land_scale).select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7']).map(to_float)
    #       #ensure we have imagery for the sensor
    if post_le7.size().getInfo() > 0 :


        #take the median
        post_le7 = post_le7.median().clip(geometry)

        #calculate nbr, ndvi and ndii
        post_le7_nbr = post_le7.normalizedDifference(['SR_B4', 'SR_B7']).select([0], ['NBR']).cast({'NBR': 'float'})
        post_le7_ndvi = post_le7.normalizedDifference(['SR_B4', 'SR_B3']).select([0], ['NDVI']).cast({'NDVI': 'float'})
        post_le7_ndii = post_le7.normalizedDifference(['SR_B4', 'SR_B5']).select([0], ['NDII']).cast({'NDII': 'float'})

        #add the bands back
        post_le72 = post_le7.addBands(post_le7_nbr).addBands(post_le7_ndvi).addBands(post_le7_ndii)

    #------------------------------------------Landsat 8 corrections


    #-------first do pre-fire

    #select bands
    pre_lc8 = lc8.filterDate(pre_start, pre_end).map(mask).map(land_scale).select(['SR_B2','SR_B3','SR_B4','SR_B5','SR_B6','SR_B7'],['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7']) .map(to_float)

    #       #ensure we have imagery for the sensor
    if pre_lc8.size().getInfo() > 0 :



        #take the median
        pre_lc8 = pre_lc8.median().clip(geometry)

        #calculate nbr, ndvi and ndii
        pre_lc8_nbr = pre_lc8.normalizedDifference(['SR_B4', 'SR_B7']).select([0], ['NBR']).cast({'NBR': 'float'})
        pre_lc8_ndvi = pre_lc8.normalizedDifference(['SR_B4', 'SR_B3']).select([0], ['NDVI']).cast({'NDVI': 'float'})
        pre_lc8_ndii = pre_lc8.normalizedDifference(['SR_B4', 'SR_B5']).select([0], ['NDII']).cast({'NDII': 'float'})

        #add the bands back
        pre_lc8 = pre_lc8.addBands(pre_lc8_nbr).addBands(pre_lc8_ndvi).addBands(pre_lc8_ndii)

        #apply the corrections

        l8_pre_corr = pre_lc8.multiply(l8_corr[1]).add(pre_lc8.pow(2).multiply(l8_corr[2])).add(pre_lc8.pow(3).multiply(l8_corr[3])).add(l8_corr[0])

    #-------now do post-fire
    #select bands
    post_lc8 = lc8.filterDate(post_start, post_end).map(mask).map(land_scale).select(['SR_B2','SR_B3','SR_B4','SR_B5','SR_B6','SR_B7'],['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7']) .map(to_float)

    #       #ensure we have imagery for the sensor
    if post_lc8.size().getInfo() > 0 :



        #take the median
        post_lc8 = post_lc8.median().clip(geometry)

        #calculate nbr, ndvi and ndii
        post_lc8_nbr = post_lc8.normalizedDifference(['SR_B4', 'SR_B7']).select([0], ['NBR']).cast({'NBR': 'float'})
        post_lc8_ndvi = post_lc8.normalizedDifference(['SR_B4', 'SR_B3']).select([0], ['NDVI']).cast({'NDVI': 'float'})
        post_lc8_ndii = post_lc8.normalizedDifference(['SR_B4', 'SR_B5']).select([0], ['NDII']).cast({'NDII': 'float'})

        #add the bands back
        post_lc8 = post_lc8.addBands(post_lc8_nbr).addBands(post_lc8_ndvi).addBands(post_lc8_ndii)

        #apply the corrections

        l8_post_corr = post_lc8.multiply(l8_corr[1]).add(post_lc8.pow(2).multiply(l8_corr[2])).add(post_lc8.pow(3).multiply(l8_corr[3])).add(l8_corr[0])

        # #          #------------------------------------------Sentinel 2 corrections, use landsat 8 coefficients


    ##-------first do pre-fire

    #select bands
    pre_sent = sent_2A.filterDate(pre_start, pre_end).map(sent_scale).select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7']).map(to_float)

    #       #ensure we have imagery for the sensor
    if pre_sent.size().getInfo() > 0 :



          #take the median
        pre_sent = pre_sent.median().clip(geometry)

        #calculate nbr, ndvi and ndii
        pre_sent_nbr = pre_sent.normalizedDifference(['SR_B4', 'SR_B7']).select([0], ['NBR']).cast({'NBR': 'float'})
        pre_sent_ndvi = pre_sent.normalizedDifference(['SR_B4', 'SR_B3']).select([0], ['NDVI']).cast({'NDVI': 'float'})
        pre_sent_ndii = pre_sent.normalizedDifference(['SR_B4', 'SR_B5']).select([0], ['NDII']).cast({'NDII': 'float'})

        #add the bands back
        pre_sent = pre_sent.addBands(pre_sent_nbr).addBands(pre_sent_ndvi).addBands(pre_sent_ndii)

        #apply the corrections

        sent_pre_corr = pre_sent.multiply(l8_corr[1]).add(pre_sent.pow(2).multiply(l8_corr[2])).add(pre_sent.pow(3).multiply(l8_corr[3])).add(l8_corr[0])

    #-------now do post-fire
    #select bands
    post_sent = sent_2A.filterDate(post_start, post_end).map(sent_scale).select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7']).map(to_float)

    #       #ensure we have imagery for the sensor
    if post_sent.size().getInfo() > 0 :



        #take the median
        post_sent = post_sent.median().clip(geometry)

        #calculate nbr, ndvi and ndii
        post_sent_nbr = post_sent.normalizedDifference(['SR_B4', 'SR_B7']).select([0], ['NBR']).cast({'NBR': 'float'})
        post_sent_ndvi = post_sent.normalizedDifference(['SR_B4', 'SR_B3']).select([0], ['NDVI']).cast({'NDVI': 'float'})
        post_sent_ndii = post_sent.normalizedDifference(['SR_B4', 'SR_B5']).select([0], ['NDII']).cast({'NDII': 'float'})

        #add the bands back
        post_sent = post_sent.addBands(post_sent_nbr).addBands(post_sent_ndvi).addBands(post_sent_ndii)

        #apply the corrections

        sent_post_corr = post_sent.multiply(l8_corr[1]).add(post_sent.pow(2).multiply(l8_corr[2])).add(post_sent.pow(3).multiply(l8_corr[3])).add(l8_corr[0])


    #try to see if image exists, if so append

    #----------------------all prefire

    #       #empty list for pre-fire, use this to combine if we have land 5, 7, 8 or sentinel
    pre_input = []

    try:
        l5_pre_corr.getInfo()
        pre_input.append(l5_pre_corr)

    except:
        pass

    try:
        pre_le72.getInfo()
        pre_input.append(pre_le72)

    except:
        pass

    try:
        l8_pre_corr.getInfo()
        pre_input.append(l8_pre_corr)

    except:
        pass

    try:
        sent_pre_corr.getInfo()
        pre_input.append(sent_pre_corr)

    except:
        pass


    #----------------------all postfire

    #         #       #empty list for post-fire, use this to combine if we have land 5, 7, 8 or sentinel
    post_input = []

    try:
        l5_post_corr.getInfo()
        post_input.append(l5_post_corr)

    except:
        pass

    try:
        post_le72.getInfo()
        post_input.append(post_le72)

    except:
        pass

    try:
        l8_post_corr.getInfo()
        post_input.append(l8_post_corr)

    except:
        pass

    try:
        sent_post_corr.getInfo()
        post_input.append(sent_post_corr)

    except:
        pass

    #return the two lists of pre input and post input
    return pre_input, post_input



In [41]:
all_territories = ee.List(grd.distinct(["Id"]).aggregate_array("Id")).getInfo()

# all_territories = [175, 189, 154, 11]

# all_territories = [175]


In [26]:
print(len(all_territories))

1


In [ ]:
years = [2012, 2014, 2015, 2020]

# years = [2012]
#months to loop through
pre_months = ['-04-01', '-05-01', '-06-01', '-07-01', '-08-01', '-09-01', '-06-01']
end_months = ['-05-01', '-06-01', '-07-01', '-08-01', '-09-01', '-10-01', '-08-31']

all_months = dict(zip(pre_months, end_months))

thresholds = [85]

for year in years:

    for territory in all_territories:


        territory22 = grd.filter(ee.Filter.eq('Id',  territory))
        
        for thresh in thresholds:



            #create output image collection
            base = 'users/spotter/fire_cnn/Russia/dnbr_check/85'
            !earthengine create collection $base

            #full pathway to images within the collection
            full_out =os.path.join(base, str(year) + '_' + str(territory))

            #check if the asset exists, if it does skip it
            try:
                exists = ee.data.getAsset(full_out)

                print(f"Image {full_out} exists")

            except:
                print(f"Image {full_out} does not exist")


                #empty list to store all months image collection, of which we will take 80th percentile
                all_months_images = []

                 #loop through all months 
                for m1, m2 in all_months.items():

                    if m1 == '-06-01' and m2 == '-08-31':

                        start_year = year - 1
                        end_year = year + 1

                    else:

                        start_year = year - 1
                        end_year = year



                    #get pre dates
                    pre_start = str(start_year) + m1
                    pre_end = str(start_year) + m2

                    #get post dates


                    post_start = str(end_year) + m1
                    post_end = str(end_year) + m2



                    #apply the function to get the pre_fire image and post_fire image
                    all_imagery = get_pre_post(pre_start, pre_end, post_start, post_end, territory22)

                    #return the pre and post fire input imagery lists
                    pre_input = all_imagery[0]
                    post_input = all_imagery[1]

                    #if the lists each are larger than 1 we have imagery
                    if (len(pre_input) >0) and (len(post_input) > 0):

                        #take the median of the image collections
                        pre_input = ee.ImageCollection(pre_input)
                        post_input = ee.ImageCollection(post_input)

                        #get median of images
                        pre_input = pre_input.median()
                        post_input= post_input.median()

                        #difference the bands
                        raw_bands = pre_input.subtract(post_input).multiply(1000)

                        b1 = raw_bands.select('SR_B1').cast({'SR_B1':'short'})
                        b2 = raw_bands.select('SR_B2').cast({'SR_B2':'short'})
                        b3 = raw_bands.select('SR_B3').cast({'SR_B3':'short'})
                        b4 = raw_bands.select('SR_B4').cast({'SR_B4':'short'})
                        b5 = raw_bands.select('SR_B5').cast({'SR_B5':'short'})
                        b6 = raw_bands.select('SR_B7').cast({'SR_B7':'short'})
                        b7 = raw_bands.select('NBR').cast({'NBR':'short'})
                        b8 = raw_bands.select('NDVI').cast({'NDVI':'short'})
                        b9 = raw_bands.select('NDII').cast({'NDII':'short'})

                        #combine all the bands
                        # raw_bands = b1.addBands(b2).addBands(b3).addBands(b4).addBands(b5).addBands(b6).addBands(b7).addBands(b8).addBands(b9)
                        raw_bands = b7.addBands(b8).addBands(b9)

                        all_months_images.append(raw_bands)



                #append to all_days vi
                raw_bands = ee.ImageCollection(all_months_images).reduce(ee.Reducer.percentile([thresh]))


                for_predict = raw_bands.select('NBR_p85')

                for_predict = for_predict.reproject(crs = 'EPSG:3413', scale = 30)
                
                #-1 is for no data during training
                # for_predict = for_predict.unmask(-1)
                                               
                #set date
                in_date = str(year) + '-01-01'
                                               
                                               # Convert the new date to milliseconds since epoch
                new_date_millis = ee.Date(in_date).millis()

                # Update the 'system:time_start' property of the image
                for_predict = for_predict.multiply(1000).toShort()
                for_predict = for_predict.set({'system:time_start' :  new_date_millis})
                                               
                                               


        
                #export image to my cloud storage
#                 task = ee.batch.Export.image.toCloudStorage(
#                                     image = for_predict.multiply(1000).toShort(),
#                                     region=territory22.geometry(), 
#                                     description='median_' + str(territory),
#                                     scale=30,
#                                     crs='EPSG:3413',
#                                     maxPixels=1e13,
#                                     bucket = 'smp-scratch')


                task = ee.batch.Export.image.toAsset(
                                  image = for_predict,
                                  region=territory22.geometry(), 
                                  description= str(territory),
                                  scale=30,
                                  crs='EPSG:3413',
                                  # crs= 'EPSG:4326',
                                  maxPixels=1e13,
                                  assetId =  full_out)
                                  # assetId =  'users/spotter/fire_cnn/mtbs_predictions/' + str(year) + '_' + str(territory22))




                task.start()

            
                 

Asset users/spotter/fire_cnn/Russia/dnbr_check/85 already exists.
Image users/spotter/fire_cnn/Russia/dnbr_check/85/2012_0 does not exist
Asset users/spotter/fire_cnn/Russia/dnbr_check/85 already exists.
Image users/spotter/fire_cnn/Russia/dnbr_check/85/2012_1 does not exist
Asset users/spotter/fire_cnn/Russia/dnbr_check/85 already exists.
Image users/spotter/fire_cnn/Russia/dnbr_check/85/2012_2 does not exist
Asset users/spotter/fire_cnn/Russia/dnbr_check/85 already exists.
Image users/spotter/fire_cnn/Russia/dnbr_check/85/2012_3 does not exist
Asset users/spotter/fire_cnn/Russia/dnbr_check/85 already exists.
Image users/spotter/fire_cnn/Russia/dnbr_check/85/2012_4 does not exist
Asset users/spotter/fire_cnn/Russia/dnbr_check/85 already exists.
Image users/spotter/fire_cnn/Russia/dnbr_check/85/2012_5 does not exist
Asset users/spotter/fire_cnn/Russia/dnbr_check/85 already exists.
Image users/spotter/fire_cnn/Russia/dnbr_check/85/2012_6 does not exist
Asset users/spotter/fire_cnn/Russi

In [ ]:
raw_bands = ee.ImageCollection(all_months_images).reduce(ee.Reducer.percentile([thresh]))

band_names = raw_bands.bandNames()

# Print the band names
print("Band names:", band_names.getInfo())

In [30]:
for_predict.getInfo()

{'type': 'Image',
 'bands': [{'id': 'NBR_p85',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': -32768,
    'max': 32767},
   'crs': 'EPSG:3413',
   'crs_transform': [30, 0, 0, 0, -30, 0]}],
 'properties': {'system:time_start': 1325376000000}}

In [29]:
't'

't'

In [39]:
# Get the acquisition date of the image
acquisition_date = for_predict.multiply(1000).toShort().date().format('YYYY-MM-dd').getInfo()

# Print the acquisition date
print('Acquisition date:', acquisition_date)

EEException: Image.date: Image does not have a 'system:time_start' property.